In [22]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain_mistralai import ChatMistralAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain_mistralai import MistralAIEmbeddings 
from langchain.vectorstores import FAISS
from my_secret_key import mistral_key

In [23]:
os.environ["MISTRAL_API_KEY"] = mistral_key

In [24]:
# llm = ChatMistralAI(temperature=0.9,max_tokens=500)
llm  = ChatMistralAI(model="mistral-large-latest")
loaders = UnstructuredURLLoader(urls=["https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"])
data=loaders.load()
len(data)

2

In [25]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200)
docs=text_splitter.split_documents(data)
print(len(docs))

17


In [26]:
embeddings = MistralAIEmbeddings()

vectorindex=FAISS.from_documents(docs,embeddings)

/Users/manojgulahe/Exercism/MyOwn/Langchain/.venv/lib/python3.9/site-packages/langchain_mistralai/embeddings.py:169: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [27]:
# Storing vector index create in local
file_path="vector_index.pkl"
with open(file_path, "wb") as f:
    pickle.dump(vectorindex, f)

TypeError: cannot pickle '_thread.RLock' object

In [10]:
file_path="vector_index.pk1"
vectorindex.save_local(file_path)

In [15]:
if(os.path.exists(file_path)):
   x=vectorindex.load_local(file_path,embeddings,allow_dangerous_deserialization=True)

In [20]:
chain=RetrievalQAWithSourcesChain.from_llm(llm=llm,retriever=x.as_retriever())


RetrievalQAWithSourcesChain(verbose=False, combine_documents_chain=MapReduceDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=ChatMistralAI(client=<httpx.Client object at 0x107952fa0>, async_client=<httpx.AsyncClient object at 0x107bcf9a0>, mistral_api_key=SecretStr('**********'), endpoint='https://api.mistral.ai/v1', model='mistral-large-latest'), output_parser=StrOutputParser(), llm_kwargs={}), reduce_documents_chain=ReduceDocumentsChain(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'summaries'], input_types={}, partial_variables={}, template='Gi

In [21]:
query = "what is the price of tiago iCNG?"
langchain.debug=True
chain({"question":query},return_only_outputs=True)

/var/folders/z3/p43c6y3969vbpl3bqsff6tz00000gn/T/ipykernel_79207/3269204299.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain({"question":query},return_only_outputs=True)


[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "what is the price of tiago iCNG?"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "The company also said it has also introduced the twin-cylinder technology on its Tiago and Tigor models.\n\nThe Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh, while the Tigor iCNG comes at a price range of Rs 7.8 lakh to Rs 8.95 lakh.\n\nTata Motors Passenger Vehicles Ltd Head-Marketing, Vinay Pant said these introductions put together will make the company's CNG line up \"appealing, holistic, and stronger than ever\".\n\nPTI\n\nTags: #Business #Companies\n\nfirst published: Aug 4, 2023 02:17 pm\n\nTop Trends\n\nAngel TaxWiproBudget newsNew Income tax 

Token indices sequence length is longer than the specified maximum sequence length for this model (1894 > 1024). Running this sequence through the model will result in indexing errors


[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "what is the price of tiago iCNG?",
  "summaries": "Content: The Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh.\nSource: https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html\n\nContent: The provided text does not mention the price of the Tiago iCNG. It only states that the twin-cylinder technology has been introduced on the Tiago and Tigor models.\nSource: https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html\n\nContent: There is no relevant text in the provided document that mentions the price of the Tiago iCNG. The text only discusses the Punch iCNG model from Tata Motors.\nSource: https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html\n\nCont

{'answer': 'The Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh.\n',
 'sources': 'https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html'}